In [1]:
from queue import Queue

from unidecode import unidecode
import networkx as nx
import spotipy #https://spotipy.readthedocs.io/en/latest/
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
# Charles profile link:
# https://open.spotify.com/user/12167056236?si=BfbG8v9wRsOTnilpjP9ibg
# spotify:user:12167056236:playlist:1ahkvZq6vSkYCmqHiPpLJn
charles_id = '12167056236'

# Jazz playlist link:
# https://open.spotify.com/user/12167056236/playlist/2ek4gj4PDBF4aspnYWfBmC?si=eq_YK6V8SdKiuhu1zj5VPA
# spotify:user:12167056236:playlist:2ek4gj4PDBF4aspnYWfBmC
jazz_id = '2ek4gj4PDBF4aspnYWfBmC'

# Bossa nova playlist link:
# https://open.spotify.com/user/12167056236/playlist/1ahkvZq6vSkYCmqHiPpLJn?si=DQT4XTRYSwWagYejOs34Ew
bossanova_id = '1ahkvZq6vSkYCmqHiPpLJn'

In [3]:
# client_credentials_manager = SpotifyClientCredentials(client_id='943e4795c796489f9243c6e7c9f64fc7', client_secret='c6c980434cb74acc9a7408e5c99618b1')
# sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
# spotify:user:12167056236:playlist:2ek4gj4PDBF4aspnYWfBmC

In [4]:
# playlists = sp.user_playlists('spotify')
# tracks = user_playlist_tracks(user = 12167056236, playlist_id=2ek4gj4PDBF4aspnYWfBmC?si=j-MSydgGR1O3KgIUea6Yzw)

In [171]:
client_credentials_manager = SpotifyClientCredentials(client_id='943e4795c796489f9243c6e7c9f64fc7', client_secret='c6c980434cb74acc9a7408e5c99618b1')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

def calc_feat_dict(results):
    artists_musics = {}
    for i, item in enumerate(results['items']):
        track = item['track']
        artists_musics[track['artists'][0]['name']] = (track['name'].split(" - ")[0])
        #print(track['id'])
        #print("%d: %s - %s" % (i, track['artists'][0]['name'], track['name'].split(" - ")[0]))
    return artists_musics

def calc_feat_tuple(results):
    artists_musics = []
    for i, item in enumerate(results['items']):
        track = item['track']
        artists_musics.append((track['artists'][0]['id'], track['artists'][0]['name'], track['id'], track['name']))
        #print((track['artists'][0]['name'], (track['name'].split(" - ")[0])))
    return artists_musics

In [182]:
dicio = {}

results = sp.user_playlist(user = charles_id, playlist_id = jazz_id, fields = "tracks,next")
tracks = results['tracks']

while tracks['next']: #passa pelos 10 conjuntos de tracks
    tracks = sp.next(tracks)
    data = calc_feat_tuple(tracks)    
    
    #inicializa o dicio com o primeiro artista deste cojunto
    for i in range(len(data)): #rodará 100 vezes
        if data[i][1] not in dicio.keys(): #trocou artista
            dicio[data[i][1]] = [data[i][2]]         
        else: #mesmo artista
            dicio[data[i][1]].append(data[i][2])

artistas = 119
print(dicio)

{'Bill Evans': ['2Zz6MC4rvI7Wp1ry0YZ6lW'], 'Bill Barron Orchestra': ['0E2dZeO85hzr2yZSBpallm', '0GYFtGcmqkONMQkZt6vWf9', '2ac5F9ZdkatCzxmDMzjbnh', '0MUA83k4ELOjidhPe272dC', '4Y0raP5UVtynQQt0cpGMgG', '2W0gLGipl36cz06K9ZvxdX', '5UAnlm9VkVlLelpmvXeD2F', '4gjJBwy7m9HKWaosbGqZGa', '6r5FtK2N8XsOD3Ipun6uMv', '10m3KAKJdD5M8Jr0we5X0g'], 'Max Greger & Orchester': ['11QY6S3Xq5d2KhmuedpKDG'], 'Lou Blackburn': ['4NsxWpK8kl1rHMLSELzQHz'], 'Charlie Rouse': ['0I11RZfKZ48lRWjcRtfY7S', '5gLxKvsW9QcBxuT29rP1gm', '0YxVWIT2nVzXVMmEOWq90x', '4xsUbxZF58lBF1RFqIUeTA', '0OdxjZWNuyuQk71Mt80M1o', '61NIluL07zAJQzyajSKZVr', '7I4vpDCeGwUiV3n8Vgnxfc', '03W8P4ZCBni3PtGlfmfedK', '5icEbO6tOjfd5EPuFW8DE6', '3SUXMVgEXlOZQJequ5T8VF', '5pJRCDfhiygQTYeYGw7URZ', '4Qbr2DGQAbWdjb2Owgpfn7', '6n1xfJWD1tEdg91ejbrEdW', '1VDd6w0FVR3yCBAvZnMV93', '0aVlJ6inWNADpmHzOqRWBK', '70nByAMbdVbmlJbRYj0ZzJ'], 'Gene Ammons': ['22bIox62GFlCNo4IDYgFRK', '6zEvOnicSnmRSRDV2fGrgk', '1ZKrs1fEIlAeHeVQStZy50', '56vomkARevYl9Fm3lXCQHl', '649ALjU8MnTDQmp